In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### Part 1 - Data Cleaning

In [2]:
address = pd.read_csv("part_one_dirty_addresses.csv", 
                      error_bad_lines=False, 
                      warn_bad_lines=False,
                      names=['0','1','2','3'])
address.head()

,0,1,2,3
0,address,NaN,NaN,NaN
1,62418 Highmeadow Dr Houston 77063,NaN,NaN,NaN
2,3004 Pamella Ct Austin 78734,NaN,NaN,NaN
3,3601 Rawhide Ct. Granbury 76049,NaN,NaN,NaN
4,62524 Magnolia Fair Way Spring 77386,NaN,NaN,NaN


In [3]:
address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       175 non-null    object
 1   1       35 non-null     object
 2   2       19 non-null     object
 3   3       6 non-null      object
dtypes: object(4)
memory usage: 5.6+ KB


In [4]:
# Combine all columns to one column and rename as address
address['address'] = address[address.columns[0:]].apply(
    lambda x: ''.join(x.dropna().astype(str)),
    axis=1
)

address = address.drop(address.columns[[0, 1, 2, 3]], axis=1)
address = address.drop(address.index[0])
address

,address
1,62418 Highmeadow Dr Houston 77063
2,3004 Pamella Ct Austin 78734
3,3601 Rawhide Ct. Granbury 76049
4,62524 Magnolia Fair Way Spring 77386
5,623351 Arlon Trail Houston 77082
...,...
170,624914 Doria Dr Austin 78728
171,3165 Pleasant Run Road Irving 75038
172,62056 Hardee Pass San Antonio 78253
173,3500 Callaghan rd 110 San antonio 78229


In [5]:
address['Number'] = 'NULL'
address['Street'] = 'NULL'
address['Apartment number'] = 'NULL'
address['City'] = 'NULL'
address['Zip'] = 'NULL'

In [6]:
# Split address to Number, Apartment number & Zip code
# Reference Link: https://docs.python.org/3.1/library/re.html
import re

for i in range(1,len(address)+1):
    txt = address['address'][i]
    temp = re.findall(r'\d+', txt)      #\d+ means one or more digit [0-9]
    res = list(map(str, temp))
    if len(res)==2:
        address['Number'][i] = res[0]
        address['Zip'][i] = res[1]
    elif len(res)==3 and ("-" in address['address'][i]):
        address['Number'][i] = res[0]
        address['Zip'][i] = res[1] + "-" + res[2]
    elif len(res)==3 and ("-" not in address['address'][i]):
        address['Number'][i] = res[0]
        address['Apartment number'][i] = res[1]
        address['Zip'][i] = res[2]
        

In [7]:
cities = pd.read_csv("part_one_texas_cities.csv")
cities.head()

,city,total_people
0,Abbott,753
1,Abernathy,"2,042"
2,Abilene,"69,773"
3,Ace,112
4,Ackerly,368


In [8]:
# Get city name from Texas_cities file if those are matched

for i in range(0,len(cities['city'])-1):
    for j in range(1,len(address['address'])+1):
        
        if re.search(cities['city'][i], address['address'][j], re.IGNORECASE):
            c = re.search(cities['city'][i], address['address'][j], re.IGNORECASE)
            
            if address['address'][j][:c.start()][len(address['address'][j][:c.start()])-1] == " " and address['address'][j][:c.start()].strip() not in address['Number'][j]:
                address['City'][j] = cities['city'][i]

In [9]:
for i in range(1,len(address['address'])):
    address['address'][i] = address['address'][i].split()
    lst = (address['Number'][i], address['Apartment number'][i], address['City'][i], address['Zip'][i])
    cleaned_lst = [ x for x in re.split('\W+', address['address'][i].strip()) if x not in lst[i] ]
    address['Street'][i] = ' '.join(cleaned_lst)

AttributeError: 'list' object has no attribute 'strip'

In [10]:
address.to_csv(r'part_one_output.csv', index = False, header=True)